In [1]:
#Initializing modules
import pandas as pd
import numpy as np
import os
import json
import copy



In [2]:
#Function
def fileList(folderPath):
    '''
    Store file name inside specified folder in array
    
    Input : folder path relative to current document
    Output : List
    '''
    fileList=[]
    for files in os.listdir(folderPath):
        fileList.append(files)
    return fileList

def bmkgPrepDataCleaning(folderPath, currentFileOut):
    getFileLists=fileList(folderPath)
    
    listInit=[]
    
    #Write result(s) header to file
    with open(currentFileOut, 'w') as fileObject:
        #fileObject.write(json.dump(listInit, indent=5))
        json.dump(listInit, fileObject, indent=5)
        
    for i in range(len(getFileLists)):
        currentData=pd.read_excel(folderPath+getFileLists[i], header=None)
        currentStationName=currentData[2][1].replace(":", "").strip()
        
        #Forward check
        #Make sure that it's not a false stop
        #Initializing data length param (to avoid header & footer miscalculation)
        #Use < and > for conditionals; index start from 0
        headerBoundary=9
        footerBoundary=len(currentData[0])-7-1 #The -1 is for normalizing index, so its started from 0

        fwdDataStatus=False
        currentResultData=[[],[]]
        for j in range(len(currentData[0])):
            if j<headerBoundary or j>footerBoundary: #Continuing iteration on xlsx header and footer
                continue
            #Check if there is a data or not
            data=currentData[1][j]
            if not pd.isna(data):
                fwdDataStatus=True
            if fwdDataStatus:
                currentResultData[0].append(currentData[0][j]) #dDate
                currentResultData[1].append(data)
                
        #Calculating Statistics
        try:
            #Count results
            countResults=len(currentResultData[1])
            #CountNaN
            countNaN=len([x for x in currentResultData[1] if pd.isna(x)])
            #countNaN=len([x for x in currentResultData[1] if x != 8888])
            #Count 8888
            count8888=len([x for x in currentResultData[1] if x==8888])
            #Count 9999
            count9999=len([x for x in currentResultData[1] if x==9999])
            #Start Date
            startDate=currentResultData[0][0]
            #End Date
            endDate=currentResultData[0][len(currentResultData[0])-1]
            #Bad result
            badResult=countNaN+count8888+count9999
            #Percentage Bad Result
            percBad=badResult/countResults*100
        except IndexError:
            #Count results
            countResults=None
            #CountNaN
            countNaN=None
            #Count 8888
            count8888=None
            #Count 9999
            count9999=None
            #Start Date
            startDate=None
            #End Date
            endDate=None
            #Bad result
            badResult=None
            #Percentage Bad Result
            percBad=None
        
        #Write result(s) to file
        resultDict={"namastasiun":currentStationName,
                    "prep":{
                        "data":currentResultData,
                        "statistics":{
                            "totalentry":countResults,
                            "startdate":startDate,
                            "enddate":endDate,
                            "countNaN":countNaN,
                            "count8888":count8888,
                            "count9999":count9999,
                            "badresults":badResult,
                            "percbadres":percBad
                        }
                    }
                   }
        with open(currentFileOut, 'r+') as fileObject:
            data=json.load(fileObject)
            data.append(resultDict)
            fileObject.seek(0)
            json.dump(data,fileObject)
            #fileObject.write(json.dumps(resultDict))
        print("Completed {}/{}. Current : {}".format(i,len(getFileLists),currentStationName))
    
        
    print("Completed")
        

In [3]:
#Folder path & initiating folder contents
bmkgDataPath='./190stasiun/'
docName=fileList(bmkgDataPath)

In [4]:
print(bmkgDataPath+docName[0])
currentData=pd.read_excel(bmkgDataPath+docName[0], header=None)
#print(currentData)

./190stasiun/Balai Besar Meteorologi, Klimatologi, dan Geofisika Wilayah I.xlsx


ImportError: Pandas requires version '2.6.0' or newer of 'openpyxl' (version '2.5.12' currently installed).

In [33]:
#Get and trim current station data regarding what file name is
print(currentData[2][1].replace(":", "").strip())

Balai Besar Meteorologi, Klimatologi, dan Geofisika Wilayah I


In [44]:
#Conditional check if NaN or not
print(pd.isna(currentData[0][1]))
currentData[1][4]

True


'Elevasi'

In [100]:
#Forward check
#Make sure that it's not a false stop
#Initializing data length param (to avoid header & footer miscalculation)
#Use < and > for conditionals; index start from 0
headerBoundary=9
footerBoundary=len(currentData[0])-7-1 #The -1 is for normalizing index, so its started from 0

fwdDataStatus=False
resultData=[[],[]]
for i in range(len(currentData[0])):
    if i<headerBoundary or i>footerBoundary: #Continuing iteration on xlsx header and footer
        continue
    #Check if there is a data or not
    data=currentData[1][i]
    if not pd.isna(data):
        fwdDataStatus=True
    if fwdDataStatus:
        resultData[0].append(currentData[0][i]) #dDate
        resultData[1].append(data)
#print(i)
#print(resultData)

#Backward check
#Make sure that it's not a false stop

#Getting insight
#Count NaN, 8888, 9999, start date, total data from start date
#Get percentage for NaN, 8888, 9999

#print(resultData)

#Count results
countResults=len(resultData[1])
#print(countResults)

#CountNaN
countNaN=len([x for x in resultData[1] if pd.isna(x)])
#print(countNaN)

#Count 8888
count8888=len([x for x in resultData[1] if x==8888])

#Count 9999
count9999=len([x for x in resultData[1] if x==9999])

#Start Date
startDate=resultData[0][0]

#End Date
endDate=resultData[0][len(resultData[0])-1]

#Bad result
badResult=countNaN+count8888+count9999

#Percentage Bad Result
percBad=badResult/countResults*100

print(percBad)
print(startDate)
print(endDate)

27.742500547405296
01-07-2007
31-12-2019


In [4]:
cleanedBMKGDATA=bmkgPrepDataCleaning(bmkgDataPath, "prepCleanedDataJSON-2.json")

Completed 0/191. Current : Balai Besar Meteorologi, Klimatologi, dan Geofisika Wilayah I
Completed 1/191. Current : Balai Besar Meteorologi, Klimatologi, dan Geofisika Wilayah IV
Completed 2/191. Current : Halim Perdana Kusuma Jakarta
Completed 3/191. Current : Stasiun Geofisika Aceh Selatan
Completed 4/191. Current : Stasiun Geofisika Aceh Selatan
Completed 5/191. Current : Pos Meteorologi Penggung
Completed 6/191. Current : Pos Pengamatan Kahang-Kahang
Completed 7/191. Current : Stasiun Geofisika Aceh Besar
Completed 8/191. Current : Stasiun Geofisika Aceh Selatan
Completed 9/191. Current : Stasiun Geofisika Alor
Completed 10/191. Current : Stasiun Geofisika Angkasapura
Completed 11/191. Current : Stasiun Geofisika Balikpapan
Completed 12/191. Current : Stasiun Geofisika Bandung
Completed 13/191. Current : Stasiun Geofisika Banjarnegara
Completed 14/191. Current : Stasiun Geofisika Deli Serdang
Completed 15/191. Current : Stasiun Geofisika Denpasar
Completed 16/191. Current : Stasiun

In [15]:
cleanedDataPath="prepCleanedDataJSON.json"
with open(cleanedDataPath, 'r') as cd:
    opened=cd.read()
    
loadDatas=json.loads(opened)
#print(loadDatas[0].keys())




for i in range(len(loadDatas)):
    namaStasiun=loadDatas[i]['namastasiun']
    totalEntry=loadDatas[i]['prep']['statistics']['totalentry']
    startDate=loadDatas[i]['prep']['statistics']['startdate']
    endDate=loadDatas[i]['prep']['statistics']['enddate']
    percBadRes=loadDatas[i]['prep']['statistics']['percbadres']
    #percBadRes=(loadDatas[i]['prep']['statistics']['count8888']+
                loadDatas[i]['prep']['statistics']['count9999'])/
                loadDatas[i]['prep']['statistics']['totalentry']*100
    try:
        print("{}% | {} - Count : {}, {} (=>) {}".format(round(percBadRes,1), namaStasiun,totalEntry,startDate,endDate))
    except TypeError:
        print("{}% | {} - Count : {}, {} (=>) {}".format(percBadRes, namaStasiun,totalEntry,startDate,endDate))


SyntaxError: invalid syntax (<ipython-input-15-fee349d7695c>, line 16)

In [32]:
i=2
rn=0
ns=loadDatas[i]['namastasiun']
sanCheck=loadDatas[i]['prep']['data'][1][rn]
dtsanCheck=loadDatas[i]['prep']['data'][0][rn]
print(ns, dtsanCheck, sanCheck)

Halim Perdana Kusuma Jakarta 02-01-1980 8


In [33]:
copyData=copy.deepcopy(loadDatas)
sortedByPerc=sorted(copyData, key=lambda x : 0 if x['prep']['statistics']['percbadres'] is None else x['prep']['statistics']['percbadres'])

#print(sortedByPerc)

for i in range(len(loadDatas)):
    namaStasiun=sortedByPerc[i]['namastasiun']
    totalEntry=sortedByPerc[i]['prep']['statistics']['totalentry']
    startDate=sortedByPerc[i]['prep']['statistics']['startdate']
    endDate=sortedByPerc[i]['prep']['statistics']['enddate']
    percBadRes=sortedByPerc[i]['prep']['statistics']['percbadres']
    
    try:
        calculateYearofRecords=round((1-percBadRes/100)*totalEntry/365,1)
        print("{}% - {} | {} - Count : {}, {} (=>) {}".format(round(percBadRes,1), calculateYearofRecords, namaStasiun,totalEntry,startDate,endDate))
    except TypeError:
        percBadRes=None
        print("{}% - {} | {} - Count : {}, {} (=>) {}".format(percBadRes, "None", namaStasiun,totalEntry,startDate,endDate))

None% - None | Balai Besar Meteorologi, Klimatologi, dan Geofisika Wilayah IV - Count : None, None (=>) None
None% - None | Stasiun Geofisika Alor - Count : None, None (=>) None
None% - None | Stasiun Geofisika Balikpapan - Count : None, None (=>) None
None% - None | Stasiun Geofisika Gorontalo - Count : None, None (=>) None
None% - None | Stasiun Geofisika Kupang - Count : None, None (=>) None
None% - None | Stasiun Geofisika Saumlaki - Count : None, None (=>) None
None% - None | Stasiun Geofisika Sumba Timur - Count : None, None (=>) None
None% - None | Stasiun Meteorologi Silangit - Count : None, None (=>) None
None% - None | Stasiun Pemantau Atmosfer Global Lore Lindu Bariri - Count : None, None (=>) None
None% - None | Stasiun Pemantau Atmosfer Global Puncak Vihara Klademak - Count : None, None (=>) None
1.9% - 46.1 | Stasiun Meteorologi Sultan Aji Muhammad Sulaiman Sepinggan - Count : 17166, 01-01-1973 (=>) 31-12-2019
2.0% - 39.2 | Stasiun Meteorologi FL Tobing - Count : 14610, 0

copyData2=copy.deepcopy(loadDatas)
sortedByPerc=sorted(copyData2, key=lambda x : 0 if x['prep']['statistics']['percbadres'] is None else x['prep']['statistics']['percbadres']/x['prep']['statistics']['totalentry']*100/100/365)

#print(sortedByPerc)

for i in range(len(loadDatas)):
    namaStasiun=sortedByPerc[i]['namastasiun']
    totalEntry=sortedByPerc[i]['prep']['statistics']['totalentry']
    startDate=sortedByPerc[i]['prep']['statistics']['startdate']
    endDate=sortedByPerc[i]['prep']['statistics']['enddate']
    percBadRes=sortedByPerc[i]['prep']['statistics']['percbadres']
    
    try:
        calculateYearofRecords=round((1-percBadRes/100)*totalEntry/365,1)
        print("{}% - {} | {} - Count : {}, {} (=>) {}".format(round(percBadRes,1), calculateYearofRecords, namaStasiun,totalEntry,startDate,endDate))
    except TypeError:
        percBadRes=None
        print("{}% - {} | {} - Count : {}, {} (=>) {}".format(percBadRes, "None", namaStasiun,totalEntry,startDate,endDate))